# Progetto: NFL Combine

**Programmazione di Applicazioni Data Intensive**<br />
Sviluppato da Leon Baiocchi e Alessandro Zanzi.

In [8]:
# setup e test librerie
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## Caso di studio: Draft result prediction
- Dati i risultati del NFL Combine tra gli anni 2009 e 2019, sfrutteremo i dati raccolti sui vari giocatori con l'intent di addestrare il modello in modo da predirre se un giocatore verrà scelto o meno al Draft.
- Da ciascun giocatore si possono estrarre delle caratteristiche:
    - età, peso, velocità sulle 40 yard, benchpress max reps, vertical jump result, ...
- Dal dataset con i risultati del NFL Combine possiamo estrarre ciascuna di queste feature sulla base delle quali il draft avrà esito positivo o negativo.

In [9]:
# pip install kaggle #decomment if already installed.

Note: you may need to restart the kernel to use updated packages.


- Utilizziamo il dataset [NFL Combine - Performance Data(2009 - 2019)](https://www.kaggle.com/datasets/redlineracer/nfl-combine-performance-data-2009-2019), in cui ogni riga contiene i dati delle performance di un dato giocatore.
- Per importare il dataset è opportuno aver installato `kaggle` e aver settato il proprio token per l'API
- Dopodichè si può passare al download dell'intero dataset in formato `csv`

In [10]:
import os.path
if not os.path.exists('NFL'+'.csv'):
    import kaggle
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('redlineracer/nfl-combine-performance-data-2009-2019', unzip=True)

In [12]:
dataset = pd.read_csv('NFL.csv', index_col=0)

In [22]:
dataset.head(6)

,Player,Age,School,Height,Weight,Sprint_40yd,Vertical_Jump,Bench_Press_Reps,Broad_Jump,Agility_3cone,Shuttle,Drafted..tm.rnd.yr.,BMI,Player_Type,Position_Type,Position,Drafted
Year,,,,,,,,,,,,,,,,,
2009,Beanie Wells\WellCh00,20.0,Ohio St.,1.8542,106.594207,4.38,85.09,25.0,325.12,NaN,NaN,Arizona Cardinals / 1st / 31st pick / 2009,31.004194,offense,backs_receivers,RB,Yes
2009,Will Davis\DaviWi99,22.0,Illinois,1.8796,118.387609,4.84,83.82,27.0,292.10,7.38,4.45,Arizona Cardinals / 6th / 204th pick / 2009,33.510073,defense,defensive_lineman,DE,Yes
2009,Herman Johnson\JohnHe23,24.0,LSU,2.0066,165.107623,5.50,NaN,21.0,NaN,NaN,NaN,Arizona Cardinals / 5th / 167th pick / 2009,41.005821,offense,offensive_lineman,OG,Yes
2009,Rashad Johnson\JohnRa98,23.0,Alabama,1.8034,92.079251,4.49,93.98,15.0,304.80,7.09,4.23,Arizona Cardinals / 3rd / 95th pick / 2009,28.312463,defense,defensive_back,FS,Yes
2009,Cody Brown\BrowCo96,22.0,Connecticut,1.8796,110.676538,4.76,92.71,26.0,304.80,7.10,4.40,Arizona Cardinals / 2nd / 63rd pick / 2009,31.327425,defense,line_backer,OLB,Yes
2009,Trevor Canfield\CanfTr20,23.0,Cincinnati,1.9304,139.252858,5.28,NaN,29.0,NaN,NaN,NaN,Arizona Cardinals / 7th / 254th pick / 2009,37.368830,offense,offensive_lineman,OG,Yes


In [23]:
dataset.tail(6)

,Player,Age,School,Height,Weight,Sprint_40yd,Vertical_Jump,Bench_Press_Reps,Broad_Jump,Agility_3cone,Shuttle,Drafted..tm.rnd.yr.,BMI,Player_Type,Position_Type,Position,Drafted
Year,,,,,,,,,,,,,,,,,
2019,Keelan Doss,NaN,UC Davis,1.8796,95.707990,NaN,NaN,8.0,NaN,NaN,NaN,NaN,27.090519,offense,backs_receivers,WR,No
2019,Xavier Crawford,NaN,Central Michigan,1.8034,84.821773,4.48,95.25,11.0,NaN,NaN,NaN,NaN,26.080939,defense,defensive_back,CB,No
2019,Jordan Brown,NaN,South Dakota St.,1.8288,91.172066,4.51,100.33,13.0,325.12,NaN,NaN,NaN,27.260221,defense,defensive_back,CB,No
2019,Corey Ballentine,NaN,Washburn,1.8034,88.904105,4.47,100.33,15.0,342.90,6.82,4.14,NaN,27.336171,defense,defensive_back,CB,No
2019,Ugo Amadi,NaN,Oregon,1.7526,90.264882,4.51,82.55,18.0,292.10,7.21,4.19,NaN,29.386861,defense,defensive_back,S,No
2019,Nasir Adderley,NaN,Delaware,1.8288,93.440028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.938336,defense,defensive_back,S,No


## Analisi variabili dataset:

- `Year`: anno in cui si è svolto il Combine training, è indice del dataset;
- `Player`: nome/id del giocatore;
- `Age`: età del giocatore;
- `School`: scuola frequentata;
- `Height`: altezza in metri(m);
- `Weight`: peso in kilogrammi(Kg);
- `Sprint_40yd`: tempo scatto sulle 40 yard in secondi;
- `Vertical_Jump`: salto verticale in centimetri(cm);
- `Bench_Press_Reps`: maggior numero di ripetizioni su panca piana con 102.1 Kg;
- `Broad_Jump`: salto in lungo in centimetri(cm);
- `Agility_3cone`: tempo migliore percorso a coni in secondi;
- `Shuttle`: scivolamento laterale in secondi;
- `Drafted..tm.rnd.yr`: il team che ha scelto il giocatore, a quale round del draft, con quale scelta(in numero) e l'anno del draft;
- `BMI`: Body Mass Index (Kg/m³);
- `Player_Type`: tipo di giocatore(difensivo o offensivo);
- `Position_Type`: classificazione ampia della posizione di gioco degli atleti;
- `Position`: posizione del giocatore;
- `Drafted`: esito del draft;
<hr />
Come si può notare anche dalla cella qui sotto, il numero di variabili è 17 mentre le istanze totali sono 3477.

In [20]:
dataset.shape

(3477, 17)

- Nell'analisi delle osservazioni ci siamo resi conto che alcune variabili possono essere scartate in quanto <i>non rilevanti</i> al fine di determinare l'esito del draft.
- Queste variabili sono le seguenti:
    - `Player`, `Age`, `Drafted..tm..rnd.yr`, `Player_Type`, `Position_Type` # TODO: Explain why
- Per quanto riguarda la variabile da predire, la nostra scelta è ricaduta su `Drafted`, una variabile discreta binaria che indica l'esito del draft, positivo o negativo, in particolare 'Yes' o 'No'.
- Iniziamo quindi a raccogliere le varie feature insieme.

In [21]:
features = [
    "School", "Height", 
    "Weight", "Sprint_40yd",
    "Vertical_Jump", "Bench_Press_Reps", 
    "Broad_Jump", "Agility_3cone",
    "Shuttle", "BMI", "Position"
]